In [37]:
# ============================================================================
# NOTEBOOK 3 : ANALYSE EXPLORATOIRE DES DONNÉES E-COMMERCE
# ============================================================================
# Objectif : Découvrir les patterns, tendances et insights cachés
# Focus : Visualisations actionables pour le business

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.float_format', '{:.2f}'.format)

print("🎯 NOTEBOOK 3 : ANALYSE EXPLORATOIRE")
print("=" * 40)
print("📊 Découverte des patterns business")
print("🔍 Insights actionnables")
print("📈 Visualisations interactives")


🎯 NOTEBOOK 3 : ANALYSE EXPLORATOIRE
📊 Découverte des patterns business
🔍 Insights actionnables
📈 Visualisations interactives


In [38]:
# ============================================================================
# 📊 CHARGEMENT ET APERÇU DES DONNÉES NETTOYÉES
# ============================================================================

# Charger les données nettoyées
df = pd.read_pickle('C:/Users/Moi/E-commerce_Marketing_Analytics/data/processed/cleaned_ecommerce_data.pkl')

# Reconversion des types (nécessaire après CSV)
#df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
#df['Customer ID'] = df['Customer ID'].astype(str)

print("📊 APERÇU DES DONNÉES NETTOYÉES")
print("=" * 35)
print(f"📏 Dimensions: {df.shape}")
print(f"📅 Période: {df['InvoiceDate'].min()} → {df['InvoiceDate'].max()}")
print(f"🌍 Pays: {df['Country'].nunique()}")
print(f"👥 Clients: {df['Customer ID'].nunique():,}")
print(f"📦 Produits: {df['StockCode'].nunique():,}")

# Aperçu des types de données
print("\n🔧 TYPES DE DONNÉES :")
print(df.dtypes)
print("\n🔧 TYPES DE DONNÉES FINAUX :")
for col in df.columns:
    print(f"  • {col}: {df[col].dtype}")


📊 APERÇU DES DONNÉES NETTOYÉES
📏 Dimensions: (1040892, 13)
📅 Période: 2009-12-01 07:45:00 → 2011-12-09 12:50:00
🌍 Pays: 43
👥 Clients: 248,949
📦 Produits: 5,305

🔧 TYPES DE DONNÉES :
Invoice                     object
StockCode                   object
Description                 object
Quantity                     int64
InvoiceDate         datetime64[ns]
Price                      float64
Customer ID                 object
Country                     object
Transaction_Type            object
Total_Amount               float64
Suspicious                    bool
Price_Outlier                 bool
Quantity_Outlier              bool
dtype: object

🔧 TYPES DE DONNÉES FINAUX :
  • Invoice: object
  • StockCode: object
  • Description: object
  • Quantity: int64
  • InvoiceDate: datetime64[ns]
  • Price: float64
  • Customer ID: object
  • Country: object
  • Transaction_Type: object
  • Total_Amount: float64
  • Suspicious: bool
  • Price_Outlier: bool
  • Quantity_Outlier: bool


In [39]:
# ============================================================================
# 💰 ANALYSE FINANCIÈRE GLOBALE
# ============================================================================

def analyze_financial_performance(df):
    """
    Analyse complète des performances financières
    """
    print("💰 ANALYSE FINANCIÈRE GLOBALE")
    print("=" * 30)
    
    # Métriques globales
    sales_data = df[df['Transaction_Type'] == 'SALE']
    returns_data = df[df['Transaction_Type'] == 'RETURN']
    
    total_revenue = sales_data['Total_Amount'].sum()
    total_returns = abs(returns_data['Total_Amount'].sum())
    net_revenue = total_revenue - total_returns
    
    print(f"💵 Chiffre d'affaires brut: ${total_revenue:,.2f}")
    print(f"↩️  Montant des retours: ${total_returns:,.2f}")
    print(f"💎 Chiffre d'affaires net: ${net_revenue:,.2f}")
    print(f"📊 Taux de retour: {(total_returns/total_revenue)*100:.1f}%")
    
    # Métriques par transaction
    avg_order_value = sales_data['Total_Amount'].mean()
    median_order_value = sales_data['Total_Amount'].median()
    
    print(f"\n📈 MÉTRIQUES PAR COMMANDE :")
    print(f"💰 Panier moyen: ${avg_order_value:.2f}")
    print(f"📊 Panier médian: ${median_order_value:.2f}")
    print(f"🛒 Nombre total de commandes: {len(sales_data):,}")
    
    # Analyse des articles
    total_items_sold = sales_data['Quantity'].sum()
    avg_items_per_order = sales_data['Quantity'].mean()
    
    print(f"\n📦 ANALYSE DES ARTICLES :")
    print(f"📊 Articles vendus: {total_items_sold:,}")
    print(f"🛒 Articles par commande: {avg_items_per_order:.1f}")
    
    return {
        'total_revenue': total_revenue,
        'net_revenue': net_revenue,
        'return_rate': (total_returns/total_revenue)*100,
        'avg_order_value': avg_order_value,
        'total_orders': len(sales_data)
    }

# Exécution de l'analyse financière
financial_metrics = analyze_financial_performance(df)


💰 ANALYSE FINANCIÈRE GLOBALE
💵 Chiffre d'affaires brut: $20,445,293.52
↩️  Montant des retours: $1,516,344.05
💎 Chiffre d'affaires net: $18,928,949.47
📊 Taux de retour: 7.4%

📈 MÉTRIQUES PAR COMMANDE :
💰 Panier moyen: $20.01
📊 Panier médian: $10.00
🛒 Nombre total de commandes: 1,021,752

📦 ANALYSE DES ARTICLES :
📊 Articles vendus: 10,907,268
🛒 Articles par commande: 10.7


In [40]:
# ============================================================================
# 📅 ANALYSE TEMPORELLE DES VENTES
# ============================================================================

def analyze_temporal_patterns(df):
    """
    Analyse des patterns temporels des ventes
    """
    print("\n📅 ANALYSE TEMPORELLE DES VENTES")
    print("=" * 30)
    
    sales_data = df[df['Transaction_Type'] == 'SALE'].copy()
    
    # Extraire les composantes temporelles
    sales_data['Date'] = sales_data['InvoiceDate'].dt.date
    sales_data['Hour'] = sales_data['InvoiceDate'].dt.hour
    sales_data['DayOfWeek'] = sales_data['InvoiceDate'].dt.day_name()
    sales_data['Month'] = sales_data['InvoiceDate'].dt.month
    sales_data['MonthName'] = sales_data['InvoiceDate'].dt.month_name()
    
    # Agrégations
    daily_sales = sales_data.groupby('Date').agg({
        'Total_Amount': 'sum',
        'Invoice': 'nunique',
        'Customer ID': 'nunique'
    }).reset_index()
    
    hourly_sales = sales_data.groupby('Hour')['Total_Amount'].sum().reset_index()
    
    monthly_sales = sales_data.groupby('MonthName').agg({
        'Total_Amount': 'sum',
        'Invoice': 'nunique'
    }).reset_index()
    
    dow_sales = sales_data.groupby('DayOfWeek')['Total_Amount'].sum().reset_index()
    
    print(f"📊 Période d'analyse: {sales_data['Date'].min()} → {sales_data['Date'].max()}")
    print(f"📈 Ventes quotidiennes moyennes: ${daily_sales['Total_Amount'].mean():,.2f}")
    
    # 🔧 LIGNE CORRIGÉE - Remplacez cette ligne :
    # print(f"🕐 Heure de pointe: {hourly_sales.loc[hourly_sales['Total_Amount'].idxmax(), 'Hour']}h")
    
    # 🔧 PAR CETTE LIGNE CORRIGÉE :
    if not hourly_sales.empty and hourly_sales['Total_Amount'].sum() > 0:
        print(f"🕐 Heure de pointe: {hourly_sales.loc[hourly_sales['Total_Amount'].idxmax(), 'Hour']}h")
    else:
        print("🕐 Heure de pointe: Aucune donnée horaire disponible")
    
    return {
        'daily_sales': daily_sales,
        'hourly_sales': hourly_sales,
        'monthly_sales': monthly_sales,
        'dow_sales': dow_sales
    }

# Exécution de l'analyse temporelle
temporal_data = analyze_temporal_patterns(df)



📅 ANALYSE TEMPORELLE DES VENTES
📊 Période d'analyse: 2009-12-01 → 2011-12-09
📈 Ventes quotidiennes moyennes: $33,849.82
🕐 Heure de pointe: 12h


In [41]:
# ============================================================================
# 🔍 DIAGNOSTIC DES DONNÉES
# ============================================================================

def diagnose_data_issues(df):
    """
    Diagnostic pour identifier les problèmes dans les données
    """
    print("\n🔍 DIAGNOSTIC DES DONNÉES")
    print("=" * 40)
    
    # Vérifications de base
    print(f"📊 Forme du DataFrame: {df.shape}")
    print(f"📋 Colonnes: {list(df.columns)}")
    print(f"🔢 Lignes totales: {len(df):,}")
    
    # Vérification Transaction_Type
    if 'Transaction_Type' in df.columns:
        print(f"\n📈 TRANSACTION_TYPE:")
        print(df['Transaction_Type'].value_counts())
        sales_count = (df['Transaction_Type'] == 'SALE').sum()
        print(f"   → Nombre de SALES: {sales_count}")
    else:
        print("❌ Colonne 'Transaction_Type' manquante!")
    
    # Vérification InvoiceDate
    if 'InvoiceDate' in df.columns:
        print(f"\n📅 INVOICEDATE:")
        print(f"   → Type: {df['InvoiceDate'].dtype}")
        print(f"   → Valeurs nulles: {df['InvoiceDate'].isnull().sum()}")
        print(f"   → Premières valeurs: {df['InvoiceDate'].head().tolist()}")
    else:
        print("❌ Colonne 'InvoiceDate' manquante!")
    
    # Vérification Total_Amount
    if 'Total_Amount' in df.columns:
        print(f"\n💰 TOTAL_AMOUNT:")
        print(f"   → Type: {df['Total_Amount'].dtype}")
        print(f"   → Valeurs nulles: {df['Total_Amount'].isnull().sum()}")
        print(f"   → Min: {df['Total_Amount'].min()}")
        print(f"   → Max: {df['Total_Amount'].max()}")
        print(f"   → Moyenne: {df['Total_Amount'].mean()}")
    else:
        print("❌ Colonne 'Total_Amount' manquante!")

# Exécutez ce diagnostic
diagnose_data_issues(df)



🔍 DIAGNOSTIC DES DONNÉES
📊 Forme du DataFrame: (1040892, 13)
📋 Colonnes: ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country', 'Transaction_Type', 'Total_Amount', 'Suspicious', 'Price_Outlier', 'Quantity_Outlier']
🔢 Lignes totales: 1,040,892

📈 TRANSACTION_TYPE:
Transaction_Type
SALE      1021752
RETURN      19140
Name: count, dtype: int64
   → Nombre de SALES: 1021752

📅 INVOICEDATE:
   → Type: datetime64[ns]
   → Valeurs nulles: 0
   → Premières valeurs: [Timestamp('2009-12-01 07:45:00'), Timestamp('2009-12-01 07:45:00'), Timestamp('2009-12-01 07:45:00'), Timestamp('2009-12-01 07:45:00'), Timestamp('2009-12-01 07:45:00')]

💰 TOTAL_AMOUNT:
   → Type: float64
   → Valeurs nulles: 0
   → Min: -168469.6
   → Max: 168469.6
   → Moyenne: 18.185315544744313


In [42]:
# ============================================================================
# 🎨 VISUALISATIONS FINANCIÈRES
# ============================================================================

def create_financial_dashboard(df, temporal_data):
    """
    Crée un dashboard financier interactif
    """
    print("\n🎨 CRÉATION DU DASHBOARD FINANCIER")
    print("=" * 35)
    
    # Configuration des subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Évolution des Ventes Quotidiennes', 'Ventes par Heure',
                       'Ventes par Jour de la Semaine', 'Ventes par Mois'),
        specs=[[{'secondary_y': True}, {'type': 'bar'}],
               [{'type': 'bar'}, {'type': 'bar'}]]
    )
    
    # 1. Évolution quotidienne
    fig.add_trace(
        go.Scatter(x=temporal_data['daily_sales']['Date'], 
                  y=temporal_data['daily_sales']['Total_Amount'],
                  name='Ventes Quotidiennes',
                  line=dict(color='blue', width=2)),
        row=1, col=1
    )
    
    # 2. Ventes par heure
    fig.add_trace(
        go.Bar(x=temporal_data['hourly_sales']['Hour'],
               y=temporal_data['hourly_sales']['Total_Amount'],
               name='Ventes par Heure',
               marker_color='lightblue'),
        row=1, col=2
    )
    
    # 3. Ventes par jour de la semaine
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    dow_ordered = temporal_data['dow_sales'].set_index('DayOfWeek').reindex(day_order).reset_index()
    
    fig.add_trace(
        go.Bar(x=dow_ordered['DayOfWeek'],
               y=dow_ordered['Total_Amount'],
               name='Ventes par Jour',
               marker_color='lightgreen'),
        row=2, col=1
    )
    
    # 4. Ventes par mois
    fig.add_trace(
        go.Bar(x=temporal_data['monthly_sales']['MonthName'],
               y=temporal_data['monthly_sales']['Total_Amount'],
               name='Ventes par Mois',
               marker_color='lightcoral'),
        row=2, col=2
    )
    
    # Mise à jour du layout
    fig.update_layout(
        title_text="📊 DASHBOARD FINANCIER E-COMMERCE",
        title_x=0.5,
        height=800,
        showlegend=False
    )
    
    pio.renderers.default='browser'
    fig.show()
    
    print("✅ Dashboard financier créé")

# Création du dashboard
create_financial_dashboard(df, temporal_data)



🎨 CRÉATION DU DASHBOARD FINANCIER
✅ Dashboard financier créé


In [44]:
# ============================================================================
# 👥 ANALYSE DES CLIENTS
# ============================================================================

def analyze_customer_behavior(df):
    """
    Analyse approfondie du comportement client
    """
    print("\n👥 ANALYSE DU COMPORTEMENT CLIENT")
    print("=" * 35)
    
    sales_data = df[df['Transaction_Type'] == 'SALE'].copy()
    
    # Métriques par client
    customer_metrics = sales_data.groupby('Customer ID').agg({
        'Total_Amount': ['sum', 'count', 'mean'],
        'Invoice': 'nunique',
        'InvoiceDate': ['min', 'max'],
        'Quantity': 'sum'
    }).round(2)
    
    customer_metrics.columns = ['Total_Spent', 'Total_Orders', 'Avg_Order_Value', 
                               'Unique_Invoices', 'First_Purchase', 'Last_Purchase', 'Items_Bought']
    
    # Période d'activité
    customer_metrics['Days_Active'] = (customer_metrics['Last_Purchase'] - customer_metrics['First_Purchase']).dt.days
    
    # Segmentation simple
    customer_metrics['Segment'] = pd.cut(customer_metrics['Total_Spent'], 
                                       bins=[0, 100, 500, 1000, float('inf')],
                                       labels=['Bronze', 'Silver', 'Gold', 'Platinum'])
    
    # Distinguer les clients invités
    customer_metrics['Customer_Type'] = customer_metrics.index.to_series().apply(
        lambda x: 'Guest' if x.startswith('GUEST_') else 'Registered'
    )
    
    print(f"📊 Total clients: {len(customer_metrics):,}")
    print(f"👤 Clients enregistrés: {(customer_metrics['Customer_Type'] == 'Registered').sum():,}")
    print(f"🎭 Clients invités: {(customer_metrics['Customer_Type'] == 'Guest').sum():,}")
    
    print(f"\n💰 RÉPARTITION PAR SEGMENT :")
    segment_dist = customer_metrics['Segment'].value_counts()
    for segment, count in segment_dist.items():
        pct = (count / len(customer_metrics)) * 100
        print(f"  {segment}: {count:,} clients ({pct:.1f}%)")
    
    print(f"\n🏆 TOP CLIENTS :")
    top_customers = customer_metrics.nlargest(5, 'Total_Spent')
    for idx, (customer_id, data) in enumerate(top_customers.iterrows(), 1):
        print(f"  {idx}. Client {customer_id}: ${data['Total_Spent']:,.2f} ({data['Total_Orders']} commandes)")
    
    return customer_metrics

# Exécution de l'analyse client
customer_data = analyze_customer_behavior(df)



👥 ANALYSE DU COMPORTEMENT CLIENT
📊 Total clients: 248,138
👤 Clients enregistrés: 5,881
🎭 Clients invités: 242,257

💰 RÉPARTITION PAR SEGMENT :
  Bronze: 233,014 clients (93.9%)
  Silver: 4,705 clients (1.9%)
  Platinum: 2,845 clients (1.1%)
  Gold: 1,435 clients (0.6%)

🏆 TOP CLIENTS :
  1. Client 18102: $580,987.04 (1040 commandes)
  2. Client 14646: $528,602.52 (3854 commandes)
  3. Client 14156: $313,437.62 (4038 commandes)
  4. Client 14911: $291,420.81 (11079 commandes)
  5. Client 17450: $244,784.25 (421 commandes)


In [46]:
# ============================================================================
# 📦 ANALYSE DES PRODUITS
# ============================================================================

def analyze_product_performance(df):
    """
    Analyse des performances produits
    """
    print("\n📦 ANALYSE DES PERFORMANCES PRODUITS")
    print("=" * 40)
    
    sales_data = df[df['Transaction_Type'] == 'SALE'].copy()
    
    # Métriques par produit
    product_metrics = sales_data.groupby(['StockCode', 'Description']).agg({
        'Quantity': 'sum',
        'Total_Amount': 'sum',
        'Invoice': 'nunique',
        'Customer ID': 'nunique'
    }).reset_index()
    
    product_metrics.columns = ['StockCode', 'Description', 'Total_Quantity', 
                              'Total_Revenue', 'Unique_Orders', 'Unique_Customers']
    
    # Calcul du prix moyen
    product_metrics['Avg_Price'] = product_metrics['Total_Revenue'] / product_metrics['Total_Quantity']
    
    # Tri par revenus
    product_metrics = product_metrics.sort_values('Total_Revenue', ascending=False)
    
    print(f"📊 Produits uniques: {len(product_metrics):,}")
    print(f"💰 Revenus totaux: ${product_metrics['Total_Revenue'].sum():,.2f}")
    
    print(f"\n🏆 TOP 10 PRODUITS (Revenus) :")
    top_products = product_metrics.head(10)
    for idx, row in top_products.iterrows():
        desc = row['Description'][:50] + '...' if pd.notna(row['Description']) and len(str(row['Description'])) > 50 else row['Description']
        print(f"  {row['StockCode']}: ${row['Total_Revenue']:,.2f} | {desc}")
    
    print(f"\n🔥 TOP 10 PRODUITS (Quantité) :")
    top_qty = product_metrics.nlargest(10, 'Total_Quantity')
    for idx, row in top_qty.iterrows():
        desc = row['Description'][:50] + '...' if pd.notna(row['Description']) and len(str(row['Description'])) > 50 else row['Description']
        print(f"  {row['StockCode']}: {row['Total_Quantity']:,} unités | {desc}")
    
    return product_metrics

# Exécution de l'analyse produit
product_data = analyze_product_performance(df)



📦 ANALYSE DES PERFORMANCES PRODUITS
📊 Produits uniques: 6,554
💰 Revenus totaux: $20,445,293.52

🏆 TOP 10 PRODUITS (Revenus) :
  M: $340,153.38 | Manual
  22423: $335,733.20 | REGENCY CAKESTAND 3 TIER
  DOT: $322,657.48 | DOTCOM POSTAGE
  85123A: $257,906.71 | WHITE HANGING HEART T-LIGHT HOLDER
  23843: $168,469.60 | PAPER CRAFT , LITTLE BIRDIE
  47566: $148,590.20 | PARTY BUNTING
  85099B: $146,151.28 | JUMBO BAG RED RETROSPOT
  84879: $129,465.61 | ASSORTED COLOUR BIRD ORNAMENT
  POST: $125,682.42 | POSTAGE
  22086: $120,145.39 | PAPER CHAIN KIT 50'S CHRISTMAS 

🔥 TOP 10 PRODUITS (Quantité) :
  84077: 106,265 unités | WORLD WAR 2 GLIDERS ASSTD DESIGNS
  85123A: 94,208 unités | WHITE HANGING HEART T-LIGHT HOLDER
  23843: 80,995 unités | PAPER CRAFT , LITTLE BIRDIE
  84879: 80,138 unités | ASSORTED COLOUR BIRD ORNAMENT
  23166: 78,033 unités | MEDIUM CERAMIC TOP STORAGE JAR
  85099B: 77,331 unités | JUMBO BAG RED RETROSPOT
  17003: 70,393 unités | BROCADE RING PURSE 
  21977: 56,116 un

In [ ]:
# ============================================================================
# 🌍 ANALYSE GÉOGRAPHIQUE
# ============================================================================

def analyze_geographic_performance(df):
    """
    Analyse des performances par pays
    """
    print("\n🌍 ANALYSE GÉOGRAPHIQUE")
    
    print("=" * 25)
    
    sales_data = df[df['Transaction_Type'] == 'SALE'].copy()
    
    # Métriques par pays
    country_metrics = sales_data.groupby('Country').agg({
        'Total_Amount': 'sum',
        'Invoice': 'nunique',
        'Customer ID': 'nunique',
        'Quantity': 'sum'
    }).reset_index()
    
    country_metrics.columns = ['Country', 'Total_Revenue', 'Total_Orders', 'Unique_Customers', 'Items_Sold']
    
    # Calculs additionnels
    country_metrics['Avg_Order_Value'] = country_metrics['Total_Revenue'] / country_metrics['Total_Orders']
    country_metrics['Revenue_per_Customer'] = country_metrics['Total_Revenue'] / country_metrics['Unique_Customers']
    
    # Tri par revenus
    country_metrics = country_metrics.sort_values('Total_Revenue', ascending=False)
    
    print(f"🌎 Pays actifs: {len(country_metrics)}")
    print(f"🏆 Top pays: {country_metrics.iloc[0]['Country']}")
    
    print(f"\n💰 TOP 10 PAYS (Revenus) :")
    top_countries = country_metrics.head(10)
    for idx, row in top_countries.iterrows():
        pct = (row['Total_Revenue'] / country_metrics['Total_Revenue'].sum()) * 100
        print(f"  {row['Country']}: ${row['Total_Revenue']:,.2f} ({pct:.1f}%)")
    
    print(f"\n👥 TOP 10 PAYS (Clients) :")
    top_customers_countries = country_metrics.nlargest(10, 'Unique_Customers')
    for idx, row in top_customers_countries.iterrows():
        print(f"  {row['Country']}: {row['Unique_Customers']:,} clients")
    
    return country_metrics

# Exécution de l'analyse géographique
geographic_data = analyze_geographic_performance(df)



🌍 ANALYSE GÉOGRAPHIQUE
🌎 Pays actifs: 43
🏆 Top pays: United Kingdom

💰 TOP 10 PAYS (Revenus) :
  United Kingdom: $17,378,389.04 (85.0%)
  EIRE: $659,371.21 (3.2%)
  Netherlands: $554,038.09 (2.7%)
  Germany: $425,019.71 (2.1%)
  France: $350,456.09 (1.7%)
  Australia: $169,283.46 (0.8%)
  Spain: $108,332.49 (0.5%)
  Switzerland: $100,707.89 (0.5%)
  Sweden: $91,869.82 (0.4%)
  Denmark: $68,580.69 (0.3%)

👥 TOP 10 PAYS (Clients) :
  United Kingdom: 244,737 clients
  EIRE: 1,614 clients
  Hong Kong: 358 clients
  Unspecified: 237 clients
  France: 223 clients
  Switzerland: 147 clients
  Portugal: 140 clients
  Germany: 107 clients
  United Arab Emirates: 88 clients
  Bahrain: 67 clients


In [47]:
# ============================================================================
# 📈 VISUALISATIONS INTERACTIVES - PERFORMANCES BUSINESS
# ============================================================================

def create_business_dashboard(df):
    """
    Dashboard interactif des performances business
    """
    print("📊 CRÉATION DU DASHBOARD BUSINESS")
    print("=" * 35)
    
    # Préparation des données pour les graphiques
    monthly_sales = df.groupby(df['InvoiceDate'].dt.to_period('M')).agg({
        'Total_Amount': 'sum',
        'Invoice': 'nunique',
        'Customer ID': 'nunique'
    }).reset_index()
    monthly_sales['InvoiceDate'] = monthly_sales['InvoiceDate'].dt.to_timestamp()
    
    # 1. GRAPHIQUE 1 : Évolution mensuelle des ventes
    fig1 = go.Figure()
    
    fig1.add_trace(go.Scatter(
        x=monthly_sales['InvoiceDate'],
        y=monthly_sales['Total_Amount'],
        mode='lines+markers',
        name='Revenus Mensuels',
        line=dict(color='#1f77b4', width=3),
        marker=dict(size=8)
    ))
    
    fig1.update_layout(
        title='📈 ÉVOLUTION DES REVENUS MENSUELS',
        xaxis_title='Mois',
        yaxis_title='Revenus ( $ )',
        template='plotly_white',
        height=500
    )
    
    fig1.show()
    
    # 2. GRAPHIQUE 2 : Top 10 pays par revenus
    top_countries = df.groupby('Country')['Total_Amount'].sum().nlargest(10).reset_index()
    
    fig2 = px.bar(
        top_countries,
        x='Total_Amount',
        y='Country',
        orientation='h',
        title='🌍 TOP 10 PAYS PAR REVENUS',
        labels={'Total_Amount': 'Revenus ( $ )', 'Country': 'Pays'}
    )
    
    fig2.update_layout(
        template='plotly_white',
        height=500,
        yaxis={'categoryorder': 'total ascending'}
    )
    
    fig2.show()
    
    # 3. GRAPHIQUE 3 : Heatmap des ventes par jour/heure
    sales_heatmap = df.pivot_table(
        values='Total_Amount',
        index=df['InvoiceDate'].dt.day_name(),
        columns=df['InvoiceDate'].dt.hour,
        aggfunc='sum'
    )
    
    # Réorganiser les jours de la semaine
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    sales_heatmap = sales_heatmap.reindex(day_order)
    
    fig3 = px.imshow(
        sales_heatmap,
        title='🕐 HEATMAP VENTES PAR JOUR/HEURE',
        labels=dict(x="Heure", y="Jour", color="Revenus ($)"),
        aspect="auto"
    )
    
    fig3.update_layout(
        template='plotly_white',
        height=400
    )
    
    fig3.show()
    
    return fig1, fig2, fig3

# Création du dashboard
dashboard_figs = create_business_dashboard(df)


📊 CRÉATION DU DASHBOARD BUSINESS


In [28]:
# ============================================================================
# 🎯 ANALYSE DES PATTERNS DE COMPORTEMENT CLIENT
# ============================================================================

def analyze_customer_behavior(df):
    """
    Analyse approfondie du comportement client
    """
    print("👥 ANALYSE COMPORTEMENT CLIENT")
    print("=" * 35)
    
    # Métriques par client
    customer_metrics = df.groupby('Customer ID').agg({
        'Total_Amount': ['sum', 'mean', 'count'],
        'Quantity': 'sum',
        'Invoice': 'nunique',
        'InvoiceDate': ['min', 'max']
    }).reset_index()
    
    # Aplatir les colonnes
    customer_metrics.columns = ['Customer_ID', 'Total_Revenue', 'Avg_Order_Value', 
                               'Total_Orders', 'Total_Items', 'Unique_Invoices', 
                               'First_Purchase', 'Last_Purchase']
    
    # Calculs additionnels
    customer_metrics['Customer_Lifetime_Days'] = (
        customer_metrics['Last_Purchase'] - customer_metrics['First_Purchase']
    ).dt.days
    
    customer_metrics['Recency_Days'] = (
        df['InvoiceDate'].max() - customer_metrics['Last_Purchase']
    ).dt.days
    
    # Segmentation simple
    customer_metrics['Revenue_Segment'] = pd.cut(
        customer_metrics['Total_Revenue'],
        bins=[0, 100, 500, 1000, float('inf')],
        labels=['Bronze', 'Silver', 'Gold', 'Platinum']
    )
    
    # Statistiques par segment
    print("💰 SEGMENTATION PAR REVENUS :")
    segment_stats = customer_metrics.groupby('Revenue_Segment').agg({
        'Total_Revenue': ['count', 'sum', 'mean'],
        'Avg_Order_Value': 'mean',
        'Total_Orders': 'mean'
    }).round(2)
    
    for segment in ['Bronze', 'Silver', 'Gold', 'Platinum']:
        if segment in segment_stats.index:
            count = segment_stats.loc[segment, ('Total_Revenue', 'count')]
            revenue = segment_stats.loc[segment, ('Total_Revenue', 'sum')]
            avg_revenue = segment_stats.loc[segment, ('Total_Revenue', 'mean')]
            avg_order = segment_stats.loc[segment, ('Avg_Order_Value', 'mean')]
            
            print(f"  🏆 {segment}: {count:,} clients | "
                  f"${revenue:,.2f} total | "
                  f"${avg_revenue:.2f} moy/client | "
                  f"${avg_order:.2f} moy/commande")
    
    # Graphique de distribution
    fig4 = px.histogram(
        customer_metrics,
        x='Total_Revenue',
        nbins=50,
        title='📊 DISTRIBUTION DES REVENUS PAR CLIENT',
        labels={'Total_Revenue': 'Revenus par Client ($)', 'count': 'Nombre de Clients'}
    )
    
    fig4.update_layout(
        template='plotly_white',
        height=400
    )
    
    fig4.show()
    
    return customer_metrics, fig4

# Analyse comportement client
customer_data, customer_fig = analyze_customer_behavior(df)


👥 ANALYSE COMPORTEMENT CLIENT
💰 SEGMENTATION PAR REVENUS :
  🏆 Bronze: 233,022 clients | $2,084,122.37 total | $8.94 moy/client | $8.91 moy/commande
  🏆 Silver: 4,710 clients | $1,078,279.34 total | $228.93 moy/client | $128.32 moy/commande
  🏆 Gold: 1,437 clients | $1,029,402.00 total | $716.35 moy/client | $176.66 moy/commande
  🏆 Platinum: 2,796 clients | $15,420,363.29 total | $5515.15 moy/client | $166.87 moy/commande


In [29]:
# ============================================================================
# 🏆 ANALYSE DES PRODUITS PERFORMANTS
# ============================================================================

def analyze_product_performance(df):
    """
    Analyse détaillée des performances produits
    """
    print("📦 ANALYSE PERFORMANCE PRODUITS")
    print("=" * 35)
    
    # Métriques par produit
    product_metrics = df.groupby(['StockCode', 'Description']).agg({
        'Total_Amount': ['sum', 'mean'],
        'Quantity': ['sum', 'mean'],
        'Customer ID': 'nunique',
        'Invoice': 'nunique'
    }).reset_index()
    
    # Aplatir les colonnes
    product_metrics.columns = ['StockCode', 'Description', 'Total_Revenue', 
                              'Avg_Revenue_per_Sale', 'Total_Quantity', 
                              'Avg_Quantity_per_Sale', 'Unique_Customers', 
                              'Unique_Orders']
    
    # Calculs additionnels
    product_metrics['Revenue_per_Customer'] = (
        product_metrics['Total_Revenue'] / product_metrics['Unique_Customers']
    )
    
    product_metrics['Repeat_Purchase_Rate'] = (
        product_metrics['Unique_Orders'] / product_metrics['Unique_Customers']
    )
    
    # Tri par revenus
    product_metrics = product_metrics.sort_values('Total_Revenue', ascending=False)
    
    print("🏆 TOP 10 PRODUITS (Revenus) :")
    top_products = product_metrics.head(10)
    for idx, row in top_products.iterrows():
        print(f"  📦 {row['StockCode']}: ${row['Total_Revenue']:,.2f} | "
              f"{row['Unique_Customers']} clients | "
              f"${row['Revenue_per_Customer']:.2f}/client")
        print(f"      📝 {row['Description'][:50]}...")
    
    print(f"\n👥 TOP 10 PRODUITS (Popularité) :")
    popular_products = product_metrics.nlargest(10, 'Unique_Customers')
    for idx, row in popular_products.iterrows():
        print(f"  📦 {row['StockCode']}: {row['Unique_Customers']} clients | "
              f"${row['Total_Revenue']:,.2f} revenus")
        print(f"      📝 {row['Description'][:50]}...")
    
    # Graphique de corrélation
    fig5 = px.scatter(
        product_metrics.head(100),  # Top 100 pour lisibilité
        x='Unique_Customers',
        y='Total_Revenue',
        size='Total_Quantity',
        hover_data=['StockCode', 'Description'],
        title='📊 CORRÉLATION POPULARITÉ vs REVENUS (Top 100)',
        labels={'Unique_Customers': 'Nombre de Clients', 'Total_Revenue': 'Revenus ($)'}
    )
    
    fig5.update_layout(
        template='plotly_white',
        height=500
    )
    
    fig5.show()
    
    return product_metrics, fig5

# Analyse performance produits
product_data, product_fig = analyze_product_performance(df)


📦 ANALYSE PERFORMANCE PRODUITS
🏆 TOP 10 PRODUITS (Revenus) :
  📦 DOT: $322,647.47 | 1429 clients | $225.79/client
      📝 DOTCOM POSTAGE...
  📦 22423: $319,187.90 | 1951 clients | $163.60/client
      📝 REGENCY CAKESTAND 3 TIER...
  📦 85123A: $248,519.61 | 1996 clients | $124.51/client
      📝 WHITE HANGING HEART T-LIGHT HOLDER...
  📦 47566: $147,351.65 | 1515 clients | $97.26/client
      📝 PARTY BUNTING...
  📦 85099B: $144,023.86 | 1529 clients | $94.19/client
      📝 JUMBO BAG RED RETROSPOT...
  📦 84879: $128,691.54 | 1176 clients | $109.43/client
      📝 ASSORTED COLOUR BIRD ORNAMENT...
  📦 22086: $118,683.69 | 1243 clients | $95.48/client
      📝 PAPER CHAIN KIT 50'S CHRISTMAS ...
  📦 POST: $110,430.41 | 592 clients | $186.54/client
      📝 POSTAGE...
  📦 79321: $81,078.32 | 537 clients | $150.98/client
      📝 CHILLI LIGHTS...
  📦 84347: $72,196.52 | 550 clients | $131.27/client
      📝 ROTATING SILVER ANGELS T-LIGHT HLDR...

👥 TOP 10 PRODUITS (Popularité) :
  📦 85123A: 1996 clie

In [30]:
# ============================================================================
# 📅 ANALYSE SAISONNALITÉ DÉTAILLÉE - NOTEBOOK 3
# ============================================================================

def seasonal_analysis_complete(df):
    """
    Analyse complète des patterns saisonniers
    """
    print("📅 ANALYSE SAISONNALITÉ COMPLÈTE")
    print("=" * 40)
    
    # Préparation des données temporelles
    df['Year'] = df['InvoiceDate'].dt.year
    df['Month'] = df['InvoiceDate'].dt.month
    df['MonthName'] = df['InvoiceDate'].dt.month_name()
    df['Quarter'] = df['InvoiceDate'].dt.quarter
    df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek
    df['DayName'] = df['InvoiceDate'].dt.day_name()
    df['Hour'] = df['InvoiceDate'].dt.hour
    df['Week'] = df['InvoiceDate'].dt.isocalendar().week
    
    # =======================================================================
    # 1. ANALYSE MENSUELLE DÉTAILLÉE
    # =======================================================================
    
    monthly_summary = df.groupby(['Year', 'Month', 'MonthName']).agg({
        'Total_Amount': ['sum', 'mean', 'count'],
        'Invoice': 'nunique',
        'Customer ID': 'nunique',
        'Quantity': 'sum'
    }).round(2)
    
    # Flatten column names
    monthly_summary.columns = ['_'.join(col).strip() for col in monthly_summary.columns]
    monthly_summary = monthly_summary.reset_index()
    
    # Calcul des métriques business
    monthly_summary['Revenue_per_Order'] = (
        monthly_summary['Total_Amount_sum'] / monthly_summary['Invoice_nunique']
    )
    monthly_summary['Revenue_per_Customer'] = (
        monthly_summary['Total_Amount_sum'] / monthly_summary['Customer ID_nunique']
    )
    
    print("📊 RÉSUMÉ MENSUEL :")
    print(monthly_summary.head())
    
    # Graphique 1: Évolution mensuelle multi-métriques
    fig1 = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Revenus Mensuels', 'Nombre de Commandes', 
                       'Clients Uniques', 'Panier Moyen'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Données pour graphique
    month_data = monthly_summary.groupby('Month').agg({
        'Total_Amount_sum': 'mean',
        'Invoice_nunique': 'mean',
        'Customer ID_nunique': 'mean',
        'Revenue_per_Order': 'mean'
    }).reset_index()
    
    # Ajout des traces
    fig1.add_trace(
        go.Scatter(x=month_data['Month'], y=month_data['Total_Amount_sum'],
                  mode='lines+markers', name='Revenus', line=dict(color='#1f77b4')),
        row=1, col=1
    )
    
    fig1.add_trace(
        go.Scatter(x=month_data['Month'], y=month_data['Invoice_nunique'],
                  mode='lines+markers', name='Commandes', line=dict(color='#ff7f0e')),
        row=1, col=2
    )
    
    fig1.add_trace(
        go.Scatter(x=month_data['Month'], y=month_data['Customer ID_nunique'],
                  mode='lines+markers', name='Clients', line=dict(color='#2ca02c')),
        row=2, col=1
    )
    
    fig1.add_trace(
        go.Scatter(x=month_data['Month'], y=month_data['Revenue_per_Order'],
                  mode='lines+markers', name='Panier Moyen', line=dict(color='#d62728')),
        row=2, col=2
    )
    
    fig1.update_layout(
        title='📅 ANALYSE SAISONNALITÉ MENSUELLE',
        height=600,
        showlegend=False,
        template='plotly_white'
    )
    
    fig1.show()
    
    # =======================================================================
    # 2. ANALYSE TRIMESTRIELLE
    # =======================================================================
    
    quarterly_data = df.groupby(['Year', 'Quarter']).agg({
        'Total_Amount': 'sum',
        'Invoice': 'nunique',
        'Customer ID': 'nunique'
    }).reset_index()
    
    # Calcul croissance trimestrielle
    quarterly_data['Quarter_Label'] = 'Q' + quarterly_data['Quarter'].astype(str) + ' ' + quarterly_data['Year'].astype(str)
    quarterly_data['Revenue_Growth'] = quarterly_data['Total_Amount'].pct_change() * 100
    
    print(f"\n📈 ANALYSE TRIMESTRIELLE :")
    print(quarterly_data)
    
    # Graphique 2: Performance trimestrielle
    fig2 = px.bar(
        quarterly_data,
        x='Quarter_Label',
        y='Total_Amount',
        title='📊 REVENUS PAR TRIMESTRE',
        labels={'Total_Amount': 'Revenus ( $ )', 'Quarter_Label': 'Trimestre'}
    )
    
    fig2.update_layout(
        template='plotly_white',
        height=400
    )
    
    fig2.show()
    
    # =======================================================================
    # 3. ANALYSE HEBDOMADAIRE
    # =======================================================================
    
    weekly_pattern = df.groupby(['DayOfWeek', 'DayName']).agg({
        'Total_Amount': ['sum', 'mean'],
        'Invoice': 'nunique',
        'Customer ID': 'nunique'
    }).reset_index()
    
    weekly_pattern.columns = ['DayOfWeek', 'DayName', 'Total_Revenue', 'Avg_Revenue',
                             'Total_Orders', 'Unique_Customers']
    
    # Réorganiser les jours
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    weekly_pattern['DayName'] = pd.Categorical(weekly_pattern['DayName'], categories=day_order, ordered=True)
    weekly_pattern = weekly_pattern.sort_values('DayName')
    
    print(f"\n📅 PATTERN HEBDOMADAIRE :")
    print(weekly_pattern)
    
    # Graphique 3: Pattern hebdomadaire
    fig3 = px.bar(
        weekly_pattern,
        x='DayName',
        y='Total_Revenue',
        title='📊 PATTERN HEBDOMADAIRE DES VENTES',
        labels={'Total_Revenue': 'Revenus ( $ )', 'DayName': 'Jour de la Semaine'}
    )
    
    fig3.update_layout(
        template='plotly_white',
        height=400
    )
    
    fig3.show()
    
    # =======================================================================
    # 4. ANALYSE HORAIRE
    # =======================================================================
    
    hourly_pattern = df.groupby('Hour').agg({
        'Total_Amount': ['sum', 'mean'],
        'Invoice': 'nunique',
        'Customer ID': 'nunique'
    }).reset_index()
    
    hourly_pattern.columns = ['Hour', 'Total_Revenue', 'Avg_Revenue', 'Total_Orders', 'Unique_Customers']
    
    print(f"\n🕐 PATTERN HORAIRE :")
    peak_hours = hourly_pattern.nlargest(5, 'Total_Revenue')
    print("Top 5 heures de pointe :")
    print(peak_hours[['Hour', 'Total_Revenue', 'Total_Orders']])
    
    # Graphique 4: Pattern horaire
    fig4 = px.line(
        hourly_pattern,
        x='Hour',
        y='Total_Revenue',
        title='📊 PATTERN HORAIRE DES VENTES',
        labels={'Total_Revenue': 'Revenus ($)', 'Hour': 'Heure'}
    )
    
    fig4.update_layout(
        template='plotly_white',
        height=400
    )
    
    fig4.show()
    
    # =======================================================================
    # 5. INSIGHTS SAISONNIERS
    # =======================================================================
    
    print(f"\n🎯 INSIGHTS SAISONNIERS CLÉS :")
    print("=" * 30)
    
    # Meilleur mois
    best_month = monthly_summary.loc[monthly_summary['Total_Amount_sum'].idxmax()]
    worst_month = monthly_summary.loc[monthly_summary['Total_Amount_sum'].idxmin()]
    
    print(f"📈 MEILLEUR MOIS : {best_month['MonthName']} {best_month['Year']}")
    print(f"   💰 Revenus: ${best_month['Total_Amount_sum']:,.2f}")
    print(f"   📦 Commandes: {best_month['Invoice_nunique']}")
    print(f"   👥 Clients: {best_month['Customer ID_nunique']}")
    
    print(f"\n📉 MOIS LE PLUS FAIBLE : {worst_month['MonthName']} {worst_month['Year']}")
    print(f"   💰 Revenus: ${worst_month['Total_Amount_sum']:,.2f}")
    print(f"   📦 Commandes: {worst_month['Invoice_nunique']}")
    
    # Meilleur jour
    best_day = weekly_pattern.loc[weekly_pattern['Total_Revenue'].idxmax()]
    worst_day = weekly_pattern.loc[weekly_pattern['Total_Revenue'].idxmin()]
    
    print(f"\n📈 MEILLEUR JOUR : {best_day['DayName']}")
    print(f"   💰 Revenus moyens: ${best_day['Total_Revenue']:,.2f}")
    print(f"   📦 Commandes: {best_day['Total_Orders']}")
    
    print(f"\n📉 JOUR LE PLUS FAIBLE : {worst_day['DayName']}")
    print(f"   💰 Revenus moyens: ${worst_day['Total_Revenue']:,.2f}")
    
    # Heures de pointe
    peak_hour = hourly_pattern.loc[hourly_pattern['Total_Revenue'].idxmax()]
    low_hour = hourly_pattern.loc[hourly_pattern['Total_Revenue'].idxmin()]
    
    print(f"\n🕐 HEURE DE POINTE : {peak_hour['Hour']}h")
    print(f"   💰 Revenus: ${peak_hour['Total_Revenue']:,.2f}")
    print(f"   📦 Commandes: {peak_hour['Total_Orders']}")
    
    print(f"\n🕐 HEURE CREUSE : {low_hour['Hour']}h")
    print(f"   💰 Revenus: ${low_hour['Total_Revenue']:,.2f}")
    
    return {
        'monthly_summary': monthly_summary,
        'quarterly_data': quarterly_data,
        'weekly_pattern': weekly_pattern,
        'hourly_pattern': hourly_pattern,
        'figures': [fig1, fig2, fig3, fig4]
    }

# Exécution de l'analyse saisonnalité
seasonal_results = seasonal_analysis_complete(df)


📅 ANALYSE SAISONNALITÉ COMPLÈTE
📊 RÉSUMÉ MENSUEL :
   Year  Month MonthName  Total_Amount_sum  Total_Amount_mean  \
0  2009     12  December         796700.16              17.81   
1  2010      1   January         622516.50              19.93   
2  2010      2  February         531288.27              18.28   
3  2010      3     March         763271.59              18.62   
4  2010      4     April         587953.24              17.47   

   Total_Amount_count  Invoice_nunique  Customer ID_nunique  Quantity_sum  \
0               44744             2330                14513        418597   
1               31238             1633                 9902        374524   
2               29060             1969                 6289        367503   
3               40989             2367                 9508        488021   
4               33653             1892                 7222        350533   

   Revenue_per_Order  Revenue_per_Customer  
0             341.93                 54.90  
1    

In [35]:
# ============================================================================
# 🔍 DÉTECTION D'ANOMALIES - NOTEBOOK 3
# ============================================================================

def detect_anomalies_complete(df):
    """
    Détection complète des anomalies dans les données e-commerce
    """
    print("🔍 DÉTECTION D'ANOMALIES COMPLÈTE")
    print("=" * 40)
    
    # Imports pour détection d'anomalies
    from scipy import stats
    from sklearn.ensemble import IsolationForest
    from sklearn.preprocessing import StandardScaler
    import numpy as np
    
    # =======================================================================
    # 1. DÉTECTION ANOMALIES TRANSACTIONS
    # =======================================================================
    
    print("💰 ANALYSE DES ANOMALIES TRANSACTIONNELLES")
    print("-" * 45)
    
    # Statistiques descriptives
    transaction_stats = df.groupby('Invoice').agg({
        'Total_Amount': 'sum',
        'Quantity': 'sum',
        'StockCode': 'nunique'
    }).reset_index()
    
    transaction_stats.columns = ['Invoice', 'Total_Order_Value', 'Total_Items', 'Unique_Products']
    
    # Calcul des seuils d'anomalie (méthode IQR)
    def detect_outliers_iqr(data, column):
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
        return outliers, lower_bound, upper_bound
    
    # Détection anomalies valeur commande
    order_anomalies, lower_order, upper_order = detect_outliers_iqr(
        transaction_stats, 'Total_Order_Value'
    )
    
    print(f"📊 STATISTIQUES VALEUR COMMANDE :")
    print(f"   Moyenne: ${transaction_stats['Total_Order_Value'].mean():.2f}")
    print(f"   Médiane: ${transaction_stats['Total_Order_Value'].median():.2f}")
    print(f"   Écart-type: ${transaction_stats['Total_Order_Value'].std():.2f}")
    print(f"   Seuil inférieur: ${lower_order:.2f}")
    print(f"   Seuil supérieur: ${upper_order:.2f}")
    print(f"   🚨 Anomalies détectées: {len(order_anomalies)} commandes")
    
    # Top anomalies par valeur
    print(f"\n🔥 TOP 10 COMMANDES ANORMALEMENT ÉLEVÉES :")
    top_anomalies = order_anomalies.nlargest(10, 'Total_Order_Value')
    for _, row in top_anomalies.iterrows():
        print(f"   📋 {row['Invoice']}: ${row['Total_Order_Value']:,.2f} | "
              f"{row['Total_Items']} items | {row['Unique_Products']} produits")
    
    # Graphique 1: Distribution avec anomalies
    fig1 = go.Figure()
    
    # Histogramme normal
    fig1.add_trace(go.Histogram(
        x=transaction_stats['Total_Order_Value'],
        nbinsx=50,
        name='Commandes normales',
        opacity=0.7
    ))
    
    # Ligne seuil supérieur
    fig1.add_vline(
        x=upper_order,
        line_dash="dash",
        line_color="red",
        annotation_text=f"Seuil anomalie:  $ {upper_order:.0f}"
    )
    
    fig1.update_layout(
        title='📊 DISTRIBUTION VALEUR COMMANDES + SEUIL ANOMALIES',
        xaxis_title='Valeur Commande ( $ )',
        yaxis_title='Fréquence',
        template='plotly_white',
        height=400
    )
    
    fig1.show()
    
    # =======================================================================
    # 2. DÉTECTION ANOMALIES PRODUITS
    # =======================================================================
    
    print(f"\n📦 ANALYSE DES ANOMALIES PRODUITS")
    print("-" * 40)
    
    # Analyse par produit
    product_anomalies = df.groupby('StockCode').agg({
        'Total_Amount': 'sum',
        'Quantity': 'sum',
        'Price': ['mean', 'std', 'min', 'max'],
        'Customer ID': 'nunique'
    }).reset_index()
    
    # Flatten columns
    product_anomalies.columns = ['StockCode', 'Total_Revenue', 'Total_Quantity', 
                                'Avg_Price', 'Price_Std', 'Min_Price', 'Max_Price', 'Unique_Customers']
    
    # Détection prix anormaux
    product_anomalies['Price_Variation'] = product_anomalies['Price_Std'] / product_anomalies['Avg_Price']
    product_anomalies['Price_Range'] = product_anomalies['Max_Price'] - product_anomalies['Min_Price']
    
    # Produits avec variations de prix suspectes
    price_anomalies = product_anomalies[
        (product_anomalies['Price_Variation'] > 1.0) |  # Variation > 100%
        (product_anomalies['Price_Range'] > product_anomalies['Avg_Price'] * 2)  # Range > 200% du prix moyen
    ].sort_values('Price_Variation', ascending=False)
    
    print(f"🚨 PRODUITS AVEC VARIATIONS DE PRIX SUSPECTES : {len(price_anomalies)}")
    print("Top 10 variations de prix :")
    for _, row in price_anomalies.head(10).iterrows():
        print(f"   📦 {row['StockCode']}: Variation {row['Price_Variation']:.2f}x | "
              f"Prix: ${row['Min_Price']:.2f} -  $ {row['Max_Price']:.2f}")
    
    # Graphique 2: Scatter plot prix vs quantité
    product_anomalies['Total_Revenue_Abs'] = product_anomalies['Total_Revenue'].abs()
    fig2 = px.scatter(
        product_anomalies.head(1000),  # Top 1000 pour lisibilité
        x='Avg_Price',
        y='Total_Quantity',
        size='Total_Revenue_Abs',
        color='Price_Variation',
        hover_data=['StockCode'],
        title='📊 DÉTECTION ANOMALIES PRODUITS : PRIX vs QUANTITÉ',
        labels={'Avg_Price': 'Prix Moyen ( $ )', 'Total_Quantity': 'Quantité Totale'},
        color_continuous_scale='Viridis'
    )
    
    fig2.update_layout(
        template='plotly_white',
        height=500
    )
    
    fig2.show()
    
    # =======================================================================
    # 3. DÉTECTION ANOMALIES CLIENTS
    # =======================================================================
    
    print(f"\n👥 ANALYSE DES ANOMALIES CLIENTS")
    print("-" * 35)
    
    # Analyse comportement client
    customer_behavior = df.groupby('Customer ID').agg({
        'Total_Amount': ['sum', 'mean', 'count'],
        'Invoice': 'nunique',
        'StockCode': 'nunique',
        'Quantity': 'sum'
    }).reset_index()
    
    customer_behavior.columns = ['Customer_ID', 'Total_Spent', 'Avg_Transaction', 
                               'Total_Transactions', 'Unique_Orders', 'Unique_Products', 'Total_Items']
    
    # Métriques clients
    customer_behavior['Avg_Items_per_Order'] = customer_behavior['Total_Items'] / customer_behavior['Unique_Orders']
    customer_behavior['Avg_Products_per_Order'] = customer_behavior['Unique_Products'] / customer_behavior['Unique_Orders']
    
    # Détection clients anormaux avec Isolation Forest
    features_for_anomaly = ['Total_Spent', 'Avg_Transaction', 'Total_Transactions', 
                           'Unique_Products', 'Avg_Items_per_Order']
    
    # Préparation des données
    X = customer_behavior[features_for_anomaly].fillna(0)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Isolation Forest
    iso_forest = IsolationForest(contamination=0.05, random_state=42)
    customer_behavior['Anomaly'] = iso_forest.fit_predict(X_scaled)
    customer_behavior['Anomaly_Score'] = iso_forest.decision_function(X_scaled)
    
    # Clients anormaux
    anomalous_customers = customer_behavior[customer_behavior['Anomaly'] == -1].sort_values('Anomaly_Score')
    
    print(f"🚨 CLIENTS ANORMAUX DÉTECTÉS : {len(anomalous_customers)}")
    print("Top 10 clients les plus anormaux :")
    for _, row in anomalous_customers.head(10).iterrows():
        print(f"   👤 Client {row['Customer_ID']}:  $ {row['Total_Spent']:,.2f} | "
              f"{row['Unique_Orders']} commandes | Score: {row['Anomaly_Score']:.3f}")
    
    # Graphique 3: Visualisation anomalies clients
    fig3 = px.scatter(
        customer_behavior,
        x='Total_Spent',
        y='Avg_Transaction',
        color='Anomaly',
        color_discrete_map={1: 'blue', -1: 'red'},
        title='📊 DÉTECTION ANOMALIES CLIENTS',
        labels={'Total_Spent': 'Dépense Totale ( $ )', 'Avg_Transaction': 'Transaction Moyenne ($)'},
        hover_data=['Customer_ID', 'Unique_Orders']
    )
    
    fig3.update_layout(
        template='plotly_white',
        height=500
    )
    
    fig3.show()
    
    # =======================================================================
    # 4. DÉTECTION ANOMALIES TEMPORELLES
    # =======================================================================
    
    print(f"\n⏰ ANALYSE DES ANOMALIES TEMPORELLES")
    print("-" * 40)
    
    # Agrégation journalière
    daily_sales = df.groupby(df['InvoiceDate'].dt.date).agg({
        'Total_Amount': 'sum',
        'Invoice': 'nunique',
        'Customer ID': 'nunique'
    }).reset_index()
    
    daily_sales.columns = ['Date', 'Daily_Revenue', 'Daily_Orders', 'Daily_Customers']
    
    # Z-score pour détection anomalies temporelles
    daily_sales['Revenue_ZScore'] = np.abs(stats.zscore(daily_sales['Daily_Revenue']))
    daily_sales['Orders_ZScore'] = np.abs(stats.zscore(daily_sales['Daily_Orders']))
    
    # Jours anormaux (Z-score > 2)
    temporal_anomalies = daily_sales[
        (daily_sales['Revenue_ZScore'] > 2) | 
        (daily_sales['Orders_ZScore'] > 2)
    ].sort_values('Revenue_ZScore', ascending=False)
    
    print(f"🚨 JOURS AVEC ACTIVITÉ ANORMALE : {len(temporal_anomalies)}")
    print("Top 10 jours anormaux :")
    for _, row in temporal_anomalies.head(10).iterrows():
        print(f"   📅 {row['Date']}:  $ {row['Daily_Revenue']:,.2f} | "
              f"{row['Daily_Orders']} commandes | Z-score: {row['Revenue_ZScore']:.2f}")
    
    # Graphique 4: Série temporelle avec anomalies
    fig4 = go.Figure()
    
    # Série normale
    fig4.add_trace(go.Scatter(
        x=daily_sales['Date'],
        y=daily_sales['Daily_Revenue'],
        mode='lines',
        name='Revenus journaliers',
        line=dict(color='blue')
    ))
    
    # Points anomalies
    fig4.add_trace(go.Scatter(
        x=temporal_anomalies['Date'],
        y=temporal_anomalies['Daily_Revenue'],
        mode='markers',
        name='Anomalies',
        marker=dict(color='red', size=8),
    ))
    
    fig4.update_layout(
        title='📊 DÉTECTION ANOMALIES TEMPORELLES',
        xaxis_title='Date',
        yaxis_title='Revenus Journaliers ( $ )',
        template='plotly_white',
        height=500
    )
    
    fig4.show()
    
    # =======================================================================
    # 5. RÉSUMÉ ANOMALIES
    # =======================================================================
    
    print(f"\n🎯 RÉSUMÉ DÉTECTION D'ANOMALIES")
    print("=" * 35)
    print(f"📊 TRANSACTIONS ANORMALES : {len(order_anomalies)} commandes")
    print(f"📦 PRODUITS SUSPECTS : {len(price_anomalies)} produits")
    print(f"👥 CLIENTS ANORMAUX : {len(anomalous_customers)} clients")
    print(f"⏰ JOURS ANORMAUX : {len(temporal_anomalies)} jours")
    
    # Pourcentages
    total_transactions = len(transaction_stats)
    total_products = len(product_anomalies)
    total_customers = len(customer_behavior)
    total_days = len(daily_sales)
    
    print(f"\n📈 POURCENTAGES D'ANOMALIES :")
    print(f"   Transactions: {len(order_anomalies)/total_transactions*100:.1f}%")
    print(f"   Produits: {len(price_anomalies)/total_products*100:.1f}%")
    print(f"   Clients: {len(anomalous_customers)/total_customers*100:.1f}%")
    print(f"   Jours: {len(temporal_anomalies)/total_days*100:.1f}%")
    
    # Recommandations
    print(f"\n💡 RECOMMANDATIONS :")
    print("🔍 ACTIONS IMMÉDIATES :")
    print("   • Vérifier les commandes > $10,000")
    print("   • Investiguer les variations de prix > 100%")
    print("   • Analyser les clients avec score anomalie < -0.5")
    print("   • Examiner les pics de ventes inexpliqués")
    
    print("🛡️ MESURES PRÉVENTIVES :")
    print("   • Alertes automatiques pour commandes > seuil")
    print("   • Validation manuelle pour variations prix > 50%")
    print("   • Monitoring comportement clients VIP")
    print("   • Système de détection temps réel")
    
    return {
        'transaction_anomalies': order_anomalies,
        'product_anomalies': price_anomalies,
        'customer_anomalies': anomalous_customers,
        'temporal_anomalies': temporal_anomalies,
        'figures': [fig1, fig2, fig3, fig4],
        'summary_stats': {
            'total_transaction_anomalies': len(order_anomalies),
            'total_product_anomalies': len(price_anomalies),
            'total_customer_anomalies': len(anomalous_customers),
            'total_temporal_anomalies': len(temporal_anomalies)
        }
    }

# Exécution de la détection d'anomalies
anomaly_results = detect_anomalies_complete(df)


🔍 DÉTECTION D'ANOMALIES COMPLÈTE
💰 ANALYSE DES ANOMALIES TRANSACTIONNELLES
---------------------------------------------
📊 STATISTIQUES VALEUR COMMANDE :
   Moyenne: $352.97
   Médiane: $192.34
   Écart-type: $1645.25
   Seuil inférieur: $-592.48
   Seuil supérieur: $987.46
   🚨 Anomalies détectées: 3972 commandes

🔥 TOP 10 COMMANDES ANORMALEMENT ÉLEVÉES :
   📋 581483: $168,469.60 | 80995 items | 1 produits
   📋 541431: $77,183.60 | 74215 items | 1 produits
   📋 574941: $52,940.94 | 14149 items | 101 produits
   📋 576365: $50,653.91 | 13956 items | 99 produits
   📋 533027: $49,844.99 | 13387 items | 111 produits
   📋 531516: $45,332.97 | 12410 items | 115 produits
   📋 493819: $44,051.60 | 25018 items | 94 produits
   📋 556444: $38,970.00 | 60 items | 1 produits
   📋 524181: $33,167.80 | 8820 items | 14 produits
   📋 567423: $31,698.16 | 12572 items | 12 produits

📦 ANALYSE DES ANOMALIES PRODUITS
----------------------------------------
🚨 PRODUITS AVEC VARIATIONS DE PRIX SUSPECTES : 11

In [36]:
# ============================================================================
# 📋 RAPPORT FINAL + RECOMMANDATIONS - NOTEBOOK 3
# ============================================================================

def generate_final_report(df, seasonal_results, anomaly_results):
    """
    Génération du rapport final avec recommandations stratégiques
    """
    print("📋 RAPPORT FINAL - ANALYSE EXPLORATOIRE")
    print("=" * 50)
    
    # =======================================================================
    # 1. SYNTHÈSE EXÉCUTIVE
    # =======================================================================
    
    print("🎯 SYNTHÈSE EXÉCUTIVE")
    print("=" * 20)
    
    # Métriques clés globales
    total_revenue = df['Total_Amount'].sum()
    total_transactions = df['Invoice'].nunique()
    total_customers = df['Customer ID'].nunique()
    total_products = df['StockCode'].nunique()
    avg_order_value = total_revenue / total_transactions
    
    # Période d'analyse
    start_date = df['InvoiceDate'].min()
    end_date = df['InvoiceDate'].max()
    analysis_period = (end_date - start_date).days
    
    print(f"📊 MÉTRIQUES BUSINESS GLOBALES")
    print(f"   💰 Chiffre d'affaires total: ${total_revenue:,.2f}")
    print(f"   📦 Nombre de commandes: {total_transactions:,}")
    print(f"   👥 Clients uniques: {total_customers:,}")
    print(f"   🛍️ Produits uniques: {total_products:,}")
    print(f"   💳 Panier moyen: ${avg_order_value:.2f}")
    print(f"   📅 Période analysée: {analysis_period} jours ({start_date.strftime('%Y-%m-%d')} → {end_date.strftime('%Y-%m-%d')})")
    
    # =======================================================================
    # 2. INSIGHTS CLÉS PAR DIMENSION
    # =======================================================================
    
    print(f"\n🔍 INSIGHTS CLÉS DÉCOUVERTS")
    print("=" * 30)
    
    # A. INSIGHTS CLIENTS
    print("👥 DIMENSION CLIENTS :")
    customer_segments = df.groupby('Customer ID').agg({
        'Total_Amount': 'sum',
        'Invoice': 'nunique'
    }).reset_index()
    
    # Segmentation clients
    top_20_customers = customer_segments.nlargest(int(len(customer_segments) * 0.2), 'Total_Amount')
    top_20_revenue = top_20_customers['Total_Amount'].sum()
    
    print(f"   💎 Top 20% clients génèrent {top_20_revenue/total_revenue*100:.1f}% du CA")
    print(f"   📈 Client le plus valuable: ${customer_segments['Total_Amount'].max():,.2f}")
    print(f"   🔄 Commandes moyennes par client: {customer_segments['Invoice'].mean():.1f}")
    
    # B. INSIGHTS PRODUITS
    print(f"\n📦 DIMENSION PRODUITS :")
    product_performance = df.groupby('StockCode').agg({
        'Total_Amount': 'sum',
        'Quantity': 'sum'
    }).reset_index()
    
    top_products = product_performance.nlargest(10, 'Total_Amount')
    top_10_revenue = top_products['Total_Amount'].sum()
    
    print(f"   🏆 Top 10 produits représentent {top_10_revenue/total_revenue*100:.1f}% du CA")
    print(f"   🎯 Produit star:  $ {product_performance['Total_Amount'].max():,.2f} de revenus")
    print(f"   📊 80/20 Rule: {(product_performance['Total_Amount'] > product_performance['Total_Amount'].quantile(0.8)).sum()} produits génèrent 80% du CA")
    
    # C. INSIGHTS TEMPORELS
    print(f"\n⏰ DIMENSION TEMPORELLE :")
    
    # Meilleurs mois (depuis seasonal_results)
    monthly_data = df.groupby(df['InvoiceDate'].dt.month).agg({
        'Total_Amount': 'sum'
    }).reset_index()
    
    best_month = monthly_data.loc[monthly_data['Total_Amount'].idxmax(), 'InvoiceDate']
    worst_month = monthly_data.loc[monthly_data['Total_Amount'].idxmin(), 'InvoiceDate']
    
    print(f"   📈 Meilleur mois: Mois {best_month} ({monthly_data['Total_Amount'].max():,.0f} $ )")
    print(f"   📉 Mois le plus faible: Mois {worst_month} ({monthly_data['Total_Amount'].min():,.0f}$)")
    
    # Jour de la semaine
    weekday_performance = df.groupby(df['InvoiceDate'].dt.dayofweek)['Total_Amount'].sum()
    best_day = weekday_performance.idxmax()
    days_names = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
    
    print(f"   🗓️ Meilleur jour: {days_names[best_day]} ({weekday_performance.max():,.0f}$)")
    
    # =======================================================================
    # 3. ANALYSE DES ANOMALIES DÉTECTÉES
    # =======================================================================
    
    print(f"\n🚨 ANOMALIES CRITIQUES DÉTECTÉES")
    print("=" * 35)
    
    # Récupération des résultats d'anomalies
    anom_stats = anomaly_results['summary_stats']
    
    print(f"📊 RÉSUMÉ DES ANOMALIES :")
    print(f"   💰 Transactions suspectes: {anom_stats['total_transaction_anomalies']}")
    print(f"   📦 Produits avec prix anormaux: {anom_stats['total_product_anomalies']}")
    print(f"   👥 Clients au comportement atypique: {anom_stats['total_customer_anomalies']}")
    print(f"   ⏰ Jours avec activité anormale: {anom_stats['total_temporal_anomalies']}")
    
    # Impact des anomalies
    transaction_anomalies = anomaly_results['transaction_anomalies']
    if len(transaction_anomalies) > 0:
        anomaly_revenue = transaction_anomalies['Total_Order_Value'].sum()
        print(f"   💸 Impact financier anomalies: ${anomaly_revenue:,.2f} ({anomaly_revenue/total_revenue*100:.1f}% du CA)")
    
    # =======================================================================
    # 4. DASHBOARD RÉCAPITULATIF
    # =======================================================================
    
    print(f"\n📊 CRÉATION DU DASHBOARD RÉCAPITULATIF")
    print("=" * 40)
    
    # Dashboard avec 4 graphiques principaux
    fig_dashboard = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            '💰 TOP 10 CLIENTS (CA)',
            '📦 TOP 10 PRODUITS (Revenus)',
            '📅 ÉVOLUTION MENSUELLE',
            '🕐 PERFORMANCE HORAIRE'
        ),
        specs=[[{"type": "bar"}, {"type": "bar"}],
               [{"type": "scatter"}, {"type": "bar"}]]
    )
    
    # 1. Top 10 clients
    top_customers = df.groupby('Customer ID')['Total_Amount'].sum().nlargest(10)
    fig_dashboard.add_trace(
        go.Bar(x=top_customers.index.astype(str), y=top_customers.values, name='Top Clients'),
        row=1, col=1
    )
    
    # 2. Top 10 produits
    top_products_viz = df.groupby('StockCode')['Total_Amount'].sum().nlargest(10)
    fig_dashboard.add_trace(
        go.Bar(x=top_products_viz.index, y=top_products_viz.values, name='Top Produits'),
        row=1, col=2
    )
    
    # 3. Évolution mensuelle
    monthly_evolution = df.groupby(df['InvoiceDate'].dt.to_period('M'))['Total_Amount'].sum()
    fig_dashboard.add_trace(
        go.Scatter(x=monthly_evolution.index.astype(str), y=monthly_evolution.values, 
                  mode='lines+markers', name='CA Mensuel'),
        row=2, col=1
    )
    
    # 4. Performance horaire
    hourly_perf = df.groupby(df['InvoiceDate'].dt.hour)['Total_Amount'].sum()
    fig_dashboard.add_trace(
        go.Bar(x=hourly_perf.index, y=hourly_perf.values, name='CA par Heure'),
        row=2, col=2
    )
    
    fig_dashboard.update_layout(
        title_text="📊 DASHBOARD RÉCAPITULATIF - ANALYSE E-COMMERCE",
        showlegend=False,
        height=800,
        template='plotly_white'
    )
    
    fig_dashboard.show()
    
    # =======================================================================
    # 5. RECOMMANDATIONS STRATÉGIQUES
    # =======================================================================
    
    print(f"\n🎯 RECOMMANDATIONS STRATÉGIQUES")
    print("=" * 35)
    
    print("🚀 RECOMMANDATIONS IMMÉDIATES (0-3 mois):")
    print("   1. 👥 CLIENTS :")
    print("      • Créer programme fidélité pour top 20% clients")
    print("      • Campagne de réactivation clients inactifs")
    print("      • Analyse détaillée des clients à forte valeur")
    print("      • Segmentation RFM pour personnalisation")
    
    print("   2. 📦 PRODUITS :")
    print("      • Optimiser stock des produits stars")
    print("      • Analyser marge des top performers")
    print("      • Cross-selling sur produits complémentaires")
    print("      • Éliminer produits low-performers")
    
    print("   3. 💰 PRICING :")
    print("      • Investiguer variations prix anormales")
    print("      • Standardiser politique tarifaire")
    print("      • Tests A/B sur pricing dynamique")
    print("      • Monitoring concurrence")
    
    print("   4. 🛡️ CONTRÔLE QUALITÉ :")
    print("      • Système d'alertes pour commandes > $10K")
    print("      • Validation manuelle transactions suspectes")
    print("      • Audit des comptes clients anormaux")
    print("      • Vérification des pics de ventes")
    
    print(f"\n📈 RECOMMANDATIONS MOYEN TERME (3-12 mois):")
    print("   1. 🎯 STRATÉGIE MARKETING :")
    print("      • Campagnes ciblées par segment client")
    print("      • Optimisation saisonnière des promotions")
    print("      • Marketing automation basé sur comportement")
    print("      • Stratégie omnicanal")
    
    print("   2. 📊 ANALYTICS AVANCÉS :")
    print("      • Modèles prédictifs de churn")
    print("      • Scoring de valeur client (CLV)")
    print("      • Recommandations produits IA")
    print("      • Forecasting des ventes")
    
    print("   3. 🏗️ INFRASTRUCTURE :")
    print("      • Dashboard temps réel")
    print("      • Système de détection anomalies automatique")
    print("      • Data pipeline optimisé")
    print("      • Reporting automatisé")
    
    # =======================================================================
    # 6. ROADMAP PROCHAINES ANALYSES
    # =======================================================================
    
    print(f"\n🗺️ ROADMAP PROCHAINES ANALYSES")
    print("=" * 30)
    
    print("📋 NOTEBOOK 4 - FEATURE ENGINEERING :")
    print("   • Création variables RFM")
    print("   • Features temporelles avancées")
    print("   • Encoding catégorielles")
    print("   • Features d'interaction")
    
    print("📋 NOTEBOOK 5 - MACHINE LEARNING :")
    print("   • Clustering clients (K-means)")
    print("   • Prédiction valeur client")
    print("   • Système de recommandation")
    print("   • Détection churn")
    
    print("📋 NOTEBOOK 6 - DÉPLOIEMENT :")
    print("   • API de prédiction")
    print("   • Dashboard interactif")
    print("   • Monitoring modèles")
    print("   • Documentation complète")
    
    # =======================================================================
    # 7. MÉTRIQUES DE PERFORMANCE
    # =======================================================================
    
    print(f"\n📈 MÉTRIQUES DE PERFORMANCE À SUIVRE")
    print("=" * 40)
    
    # Calcul des KPIs actuels
    monthly_growth = df.groupby(df['InvoiceDate'].dt.to_period('M'))['Total_Amount'].sum()
    if len(monthly_growth) > 1:
        last_month_growth = ((monthly_growth.iloc[-1] - monthly_growth.iloc[-2]) / monthly_growth.iloc[-2] * 100)
    else:
        last_month_growth = 0
    
    print("🎯 KPIs ACTUELS :")
    print(f"   💰 CA mensuel moyen: ${monthly_growth.mean():,.2f}")
    print(f"   📈 Croissance dernier mois: {last_month_growth:.1f}%")
    print(f"   👥 Clients actifs/mois: {total_customers/(analysis_period/30):.0f}")
    print(f"   🛒 Commandes/jour: {total_transactions/(analysis_period):.1f}")
    print(f"   💳 Panier moyen: ${avg_order_value:.2f}")
    
    print(f"\n🎯 KPIs À MONITORER :")
    print("   • Taux de rétention client")
    print("   • Customer Lifetime Value (CLV)")
    print("   • Taux de conversion")
    print("   • Fréquence d'achat")
    print("   • Marge brute par segment")
    
    # Sauvegarde des résultats
    report_summary = {
        'total_revenue': total_revenue,
        'total_transactions': total_transactions,
        'total_customers': total_customers,
        'avg_order_value': avg_order_value,
        'analysis_period': analysis_period,
        'anomalies_detected': anom_stats,
        'top_customers': top_customers.to_dict(),
        'top_products': top_products_viz.to_dict(),
        'monthly_growth': last_month_growth
    }
    
    print(f"\n✅ RAPPORT FINAL GÉNÉRÉ AVEC SUCCÈS !")
    print("=" * 35)
    
    return report_summary, fig_dashboard

# Génération du rapport final
final_report, dashboard_fig = generate_final_report(df, seasonal_results, anomaly_results)

# =======================================================================
# CONCLUSION NOTEBOOK 3
# =======================================================================

print("\n" + "="*60)
print("🎉 NOTEBOOK 3 - ANALYSE EXPLORATOIRE TERMINÉ !")
print("="*60)

print(f"\n📊 RÉSUMÉ DE CE QUE NOUS AVONS ACCOMPLI :")
print("   ✅ Analyse exploratoire complète")
print("   ✅ Visualisations avancées (15+ graphiques)")
print("   ✅ Analyse saisonnalité détaillée")
print("   ✅ Détection d'anomalies multi-dimensionnelle")
print("   ✅ Insights business actionnables")
print("   ✅ Recommandations stratégiques")
print("   ✅ Dashboard récapitulatif")
print("   ✅ Roadmap pour la suite")

print(f"\n🚀 PROCHAINE ÉTAPE : NOTEBOOK 4 - FEATURE ENGINEERING")
print("   🎯 Objectif: Préparer les données pour le ML")
print("   🛠️ Techniques: RFM, Variables temporelles, Encoding")
print("   📊 Résultat: Dataset prêt pour modélisation")

print(f"\n💾 DONNÉES PRÊTES POUR LA SUITE :")
print(f"   • Dataset nettoyé: {len(df)} lignes")
print(f"   • Variables créées: {len(df.columns)} colonnes")
print(f"   • Anomalies identifiées: {sum(final_report['anomalies_detected'].values())}")
print(f"   • Insights business: 20+ recommandations")




📋 RAPPORT FINAL - ANALYSE EXPLORATOIRE
🎯 SYNTHÈSE EXÉCUTIVE
📊 MÉTRIQUES BUSINESS GLOBALES
   💰 Chiffre d'affaires total: $18,928,949.47
   📦 Nombre de commandes: 53,628
   👥 Clients uniques: 248,949
   🛍️ Produits uniques: 5,305
   💳 Panier moyen: $352.97
   📅 Période analysée: 738 jours (2009-12-01 → 2011-12-09)

🔍 INSIGHTS CLÉS DÉCOUVERTS
👥 DIMENSION CLIENTS :
   💎 Top 20% clients génèrent 98.5% du CA
   📈 Client le plus valuable: $570,380.61
   🔄 Commandes moyennes par client: 1.2

📦 DIMENSION PRODUITS :
   🏆 Top 10 produits représentent 9.2% du CA
   🎯 Produit star:  $ 322,647.47 de revenus
   📊 80/20 Rule: 1061 produits génèrent 80% du CA

⏰ DIMENSION TEMPORELLE :
   📈 Meilleur mois: Mois 11 (2,872,964 $ )
   📉 Mois le plus faible: Mois 2 (1,028,339$)
   🗓️ Meilleur jour: Jeudi (3,902,397$)

🚨 ANOMALIES CRITIQUES DÉTECTÉES
📊 RÉSUMÉ DES ANOMALIES :
   💰 Transactions suspectes: 3972
   📦 Produits avec prix anormaux: 1170
   👥 Clients au comportement atypique: 12448
   ⏰ Jours avec a